In [3]:
import sqlite3, random, pathlib
from search import search_library
from dotenv import load_dotenv
from openai import OpenAI, OpenAIError
from pydantic import BaseModel, Field, ValidationError
import os
from openai import OpenAI
from pydantic import BaseModel
import json
import time
import random
from helpers.sentence_splitter import split_sentences
from time import sleep
from tqdm import tqdm  # optional: pip install tqdm



SYS_PROMPT_PARSE_ELEMENTS = ("Read the following text and separate each element of the suggested story framework into a object with the name and text.\n"
        "You should return a list of objects containing each story element in a given structure.\n"
        "Do not add or remove text; your job is to parse and deconstruct the text string into the given structure.\n"
        "Treat every line that begins with a number followed by “.” and a space (e.g. “1. ”) or **Element** as the NAME of a new element."
        "Capture all following lines up to—but not including—the next such line as the TEXT for that element.")

SYS_PROMPT_PARSE_ELEMENTS_MODIFIED = ("Read the following text and separate each element of the suggested story framework into a object with the name and text.\n"
        "You should return a list of objects containing each story element in a given structure.\n"
        "Do not add or remove text; your job is to parse and deconstruct the text string into the given structure.\n"
        "Treat every line that begins with 'Element:' followed by a word/phrase as the NAME of a new element."
        "Capture all following lines up to—but not including—the next such line as the TEXT for that element.")

SYSTEM_PROMPT_IMAGINE = """
Your task is to take a human imaginative directive and translate it into a coherent, structured list of domains, technologies, and disciplines that could be explored to realize or approximate that imaginative directive.
The goal is to break down the directive into actionable or researchable avenues across science, technology, psychology, education, or other relevant fields.
Each entry should be self-contained, meaning it should be understandable on its own, and clearly tied back to the core imaginative directive.
"""


ELEM_FUNC = {
    "type": "function",
    "function": {
        "name": "element_function",
        "description": "Read the following text and separate each element of the suggested story framework into a object with the name and text. \
        You should return a list of objects containing each story element in a given structure. \
        Do not add or remove text; your job is to parse and deconstruct the text string into the given structure.",
        "parameters": {
            "type": "object",
            "properties": {
                "element": {
                    "type": "array",
                    "items": {
                        "type": "object",
                        "properties": {
                            "name": { "type": "string" },
                            "text": { "type": "string" }
                        },
                        "required": ["name", "text"]
                    }
                }
            },
            "required": ["element"]
        }
    }
}




def ask(sys_msg, usr_msg):
    resp = client.chat.completions.create(
        model=OPENAI_MODEL,
        messages=[
            {"role": "system", "content": sys_msg},
            {"role": "user", "content": usr_msg}
        ]
    )
    return resp.choices[0].message.content

def ask_func(sys_msg, usr_msg, FUNC):
    resp = client.chat.completions.create(
            model=OPENAI_MODEL,
            messages=[{"role":"system","content":sys_msg}, {"role":"user","content":usr_msg}],
            tools=[FUNC], tool_choice="auto", temperature=0.5
            )
    return resp


def print_object_fields(obj: dict, index: int = 1, fields: list[str] = ["name", "text"]) -> None:
    print(f"{index} -")
    for field in fields:
        print(f"{field}: {obj.get(field, '[Missing]')}")


def parse_function_response(
    response,
    obj_key: str = "element",
    fields: list[str] = ["name", "text"],
    display: bool = True
) -> list[dict]:
    """Parse tool call response and return a list of objects with optional printing."""
    args_raw = response.choices[0].message.tool_calls[0].function.arguments
    objects = json.loads(args_raw).get(obj_key, [])

    result = []
    for i, obj in enumerate(objects, 1):
        if display:
            print_object_fields(obj, index=i, fields=fields)
        result.append(obj)

    return result

def process_response(response, sys_msg=SYS_PROMPT_PARSE_ELEMENTS, ELEM_FUNC=ELEM_FUNC):
    responses_func = ask_func(sys_msg, response, ELEM_FUNC)
    responses_raw = parse_function_response(responses_func, display=False)
    responses = []
    for response_raw in responses_raw:
        n, t = response_raw['name'], response_raw['text']
        responses.append({n: t})
    return responses


load_dotenv()
client = OpenAI()
OPENAI_MODEL = os.getenv("OPENAI_MODEL", "gpt-4o")

DB_FILE = "library_temp.sqlite"
MAX_RESULTS = 100




In [5]:
imagination = {}
story_space = {}

In [6]:
fantasia = "I imagine instantly learning any topic."

sys_msg = SYSTEM_PROMPT_IMAGINE
usr_msg = fantasia

seeds_resp = ask(sys_msg, usr_msg)
print(seeds_resp)

sys_msg = SYS_PROMPT_PARSE_ELEMENTS
seeds_func = ask_func(sys_msg, seeds_resp, ELEM_FUNC)
seeds_raw = parse_function_response(seeds_func, display=False)
seeds = []
for seed_raw in seeds_raw:
    n, t = seed_raw['name'], seed_raw['text']
    seeds.append({n: t})
    
imagination[fantasia] = seeds

To realize or approximate the imaginative directive of "instantly learning any topic," we can break this down into several domains, technologies, and disciplines that could be explored:

1. **Neuroscience:**
   - **Neural Encoding and Decoding:** Study the processes involved in how the brain encodes, stores, and retrieves information. Understand neural plasticity and how it can be optimized for faster learning.
   - **Brain-Computer Interfaces (BCIs):** Explore the development of devices that can directly interact with the brain to enhance or accelerate learning processes.

2. **Artificial Intelligence and Machine Learning:**
   - **Personalized Learning Algorithms:** Develop AI systems that tailor educational content to individual learning styles and speeds, optimizing the learning process.
   - **Knowledge Graphs:** Implement AI systems that use vast networks of interlinked information to provide comprehensive answers instantly, simulating an understanding.

3. **Virtual and Augmente

In [25]:
SYSTEM_PROMPT_DOMAIN_CONSTRUCTION = """
You are a conceptual analyst and domain cartographer. Your job is to explore a given domain and return a structured breakdown of the key conceptual dimensions that define the space. These are the major axes, clusters, or organizing categories that the field naturally decomposes into — such as themes, frameworks, methodologies, subfields, or core problems.

Each dimension should be:

Conceptually distinct
Semantically meaningful
Representative of a broad cluster of activity or inquiry in the domain
Useful for organizing knowledge, generating ideas, or navigating research
Avoid superficial groupings or purely keyword-based associations. Aim for a thoughtful, high-level decomposition based on meaning and structure.
"""

USER_PROMPT_DOMAIN_CONSTRUCTION = """
Explore the domain: {domain}.

This domain has been identified from the following exploration of conceptual space: {localization}

Return a structured list of the main conceptual dimensions in this domain. For each dimension, give:

Name – a concise label
Description – 1–2 sentence explanation
Examples – 2–5 representative ideas, technologies, theories, or questions that fall within this dimension
"""



def construct_domain_space(dim_dir, localization):
    # what the dimensions in story space? 
    ## If you have data, constrain the construction of story space. If you don't have data simulate possible datasets. Organize construction around those datasets. 
    ### as you construct dimensions, construct dimensions completely. the axes within a space should be complete.
    sys_msg = SYSTEM_PROMPT_DOMAIN_CONSTRUCTION
    usr_msg = USER_PROMPT_DOMAIN_CONSTRUCTION.format(domain=dim_dir, localization=localization)
    domain_dimensions = ask(sys_msg, usr_msg)
    dom_dim_parsed = process_response(domain_dimensions)
    dim_dir = list(dom_dim_parsed[0].keys())[0]
    return dim_dir

    



    # What are the paths that you can walk along story space?
    ## if you have data, then constrain the paths that you walk along.  If you don't have data simulate possible datasets. Organize construction around those datasets. 
    ### within a given dimensions construct possible complete trajectories within a space. 
    #### if you have data, constain possible complete trajectories of a space.

    

    
    


In [27]:
for fantasia, dimensions in list(imagination.items())[0:1]:
    print(fantasia)
    for dimension in dimensions[0:1]:
        dim_dir, dim_text = list(dimension.items())[0]

        localization = f"ORIGINAL DOMAIN:\n{dim_dir}\n\n"

        # what are the dimensions of story-telling along this dimension
        for _ in range(10):
            dim_dir = construct_domain_space(dim_dir, localization)
            localization = f"SUB DOMAIN:\n{dim_dir}\n\n"
            print(dim_dir)
        

        # loop dimensions and see what happens?

        # walk one path completely.
        ## if you have data, constrain possible complete one path trajectories.  If you don't have data simulate possible datasets. Organize construction around those datasets. 
        # construct many paths at once.
        ## If you have data, constrain possible complete trajectories.

        #element_text = direction_to_string(element)
        #sys_msg = SYSTEM_PROMPT_CONSTRUCTION_DIMENSIONS
        #usr_msg = USER_PROMPT_CONSTRUCTION_DIMENSIONS.format(fantasia=fantasia, direction_text=direction_to_string(direction))

I imagine instantly learning any topic.
Neuroanatomy and Neurophysiology
1. Structural Organization
1. Hierarchical Structures
1. Taxonomic Hierarchies
1. **Foundational Principles**
Foundational Principles
1. Epistemological Foundations
Nature of Knowledge
Epistemology
Nature of Knowledge


In [12]:
# what is your fantasia?

# - loop start
# what the dimensions in story space? 
## If you have data, constrain the construction of story space. If you don't have data simulate possible datasets. Organize construction around those datasets. 
### as you construct dimensions, construct dimensions completely. the axes within a space should be complete.
# What are the paths that you can walk along story space?
## if you have data, then constrain the paths that you walk along.  If you don't have data simulate possible datasets. Organize construction around those datasets. 
### within a given dimensions construct possible complete trajectories within a space. 
#### if you have data, constain possible complete trajectories of a space.
# walk one path completely.
## if you have data, constrain possible complete one path trajectories.  If you don't have data simulate possible datasets. Organize construction around those datasets. 
# construct many paths at once.
## If you have data, constrain possible complete trajectories.
# - loop end

# you have your fantasia.


